In [ ]:
from google.colab import drive
import os

# gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

# # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# To run from the common drive:
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/genius_and_wasabi/'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pandas bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 30.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039285 sha256=fa9b86115e3779a3bd5e511f335912fda57aa75465d82ae4d1591edcb5d0dc4c
  Stored in di

In [ ]:
!pip install pandas nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('wordnet') # to use wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# from nltk.corpus import wordnet as wn
# # Find synsets for a word
# synsets = wn.synsets('dog')
# print(synsets)

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]


In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from umap import UMAP

# set path of the file
csv_file_path = 'concatenated_chunks.csv'
df = pd.read_csv(dataset_path + csv_file_path)

# **Data Preprocessing**
This involves removing the explicit song structure from the lyrics column and initialising the CountVectorizer so that stop word removal is handled internally by BERTopic

In [ ]:
# Removing explicit song structure as it is not important information
import re

def remove_explicit_song_structure(lyrics):
  pattern = r'\[.+?\]'
  cleaned_lyrics = re.sub(pattern, '', lyrics)
  return cleaned_lyrics

df['cleaned_lyrics'] = df['lyrics'].astype(str).apply(remove_explicit_song_structure)
df.head()

,Unnamed: 0.1,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics
0,0,0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",en,Male,"\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa ..."
1,1,1,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,en,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197..."
2,2,2,Family Ties,rap,Cam'ron,2004,41960,"{""Cam\\'ron"",""Lady Wray""}","[Verse 1: Cam'ron]\nKilla, Dipset\nMan I spit ...",en,Male,"\nKilla, Dipset\nMan I spit that pimp talk, yo..."
3,3,3,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,en,Male,\nAy yo you wonder who I are\nI guzzle up at t...
4,4,4,Lord You Know,rap,Cam'ron,2004,11882,"{""Cam\\'ron"",""Juelz Santana"",Jaheim}","[Chorus: Jaheim]\nNow Lord you know, just how ...",en,Male,"\nNow Lord you know, just how hard I try\nTo l..."


In [ ]:
# Initializing the CountVectorizer with English stop words to pass as a parameter to BERTopic
count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words="english")
# Initialising the UMAP constructor to set the random seed - this is so the results are reproducible

umap = UMAP(n_neighbors=15,
            n_components=5,
            min_dist=0.0,
            metric='cosine',
            low_memory=False,
            random_state=42)

In [ ]:
# Model path
# model_path = '/content/gdrive/MyDrive/Bertopic/shared_work/mymodel'

# Model path for common drive

model_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_' + csv_file_path.split('/')[-1].split('.')[0]

stratify = 1
if stratify:
  model_path += '_stratified'


print(model_path)

/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified


In [ ]:
# Importing the entire dataset and performing the necessary transformations

# df = pd.read_csv(dataset_path + csv_file_path)
df['cleaned_lyrics'] = df['lyrics'].astype(str).apply(remove_explicit_song_structure)
df['has_lyrics'] = ~df['lyrics'].isna() # flag the rows that have lyrics
documents = df[df['has_lyrics']]['cleaned_lyrics'].astype(str).tolist()

In [ ]:
loaded_model = BERTopic.load(model_path)
generated_topics, prob = loaded_model.transform(documents)
print(generated_topics)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[  7   0 434 ...  66  98 139]


***To measure gender bias per topic in a dataset using BERTopic and WEAT (Word Embedding Association Test), you need to follow a series of steps. These include topic modeling with BERTopic, creating target and attribute word sets for WEAT, and then performing the WEAT analysis for each topic. To integrate WEAT analysis with BERTopic, you need a separate word embeddings model.***

Step 1 Analyzing topic distribution across genders

In [ ]:
# Checking what kind of labels are returned
topic_labels = loaded_model.generate_topic_labels()
topic_labels

['-1_love_im_dont',
 '0_nigga_niggas_bitch',
 '1_jesus_praise_lord',
 '2_body_girl_baby',
 '3_dance_funky_dance dance',
 '4_ayy ayy_change_long sentiment',
 '5_christmas_santa_year',
 '6_wi_ye_sae',
 '7_mi_dem_yuh',
 '8_shes_shell_borderlines',
 '9_money_money money_dub dub',
 '10_love_make love_baby',
 '11_tears_heart_wish',
 '12_blues_worried_blues blues',
 '13_id_world world_wouldnt',
 '14_shes_shes got_cameron',
 '15_sea_sail_ocean',
 '16_mama_mama dont_woman woman',
 '17_black_white_white man',
 '18_feature_remix_demo',
 '19_shadow_light_moon',
 '20_christ_law_unto',
 '21_fool_fools_im fool',
 '22_smoke_weed_marijuana',
 '23_vote_skeleton_reron',
 '24_raining_rain_raining raining',
 '25_oliver_sea_moira',
 '26_la la_la_lalala',
 '27_johanna_mortimer_thou',
 '28_spring_summer_autumn',
 '29_album_songs_song',
 '30_angel_youre angel_angel angel',
 '31_sudden felt_moving sound_felt body',
 '32_rattle roll_shake rattle_eat',
 '33_heartache_bah_bah bah',
 '34_drink_drink drink_beer',
 '

In [ ]:
# Percentage of outliers

list(generated_topics).count(-1)/len(generated_topics) * 100

1.599470191776439

In [ ]:
# Visualizing the intertopic distance

# loaded_model.visualize_topics()

In [ ]:
# Visualizing the topic similarities

# loaded_model.visualize_heatmap()

In [ ]:
# loaded_model.visualize_barchart()

###**Outlier Reduction**

- Initially done using the default method (c-Tf-Idf) provided by BERTopic
- The topic model resulting from the stratified sample contains only ~1.5% of outliers - hence outlier reduction is **not** done for the final model and simply ignored during analysis

In [ ]:
# new_topics = loaded_model.reduce_outliers(documents, generated_topics)
# print(new_topics)
# loaded_model.update_topics(docs=documents, topics=new_topics)

###**(Optional) Topic Reduction***

An optional topic reduction step (initially done to explore various BERTopic functionalities). One of the advantages of doing so is that you can decide the number of topics after knowing how many are created. It is difficult to predict before training your model how many topics that are in your documents and how many will be extracted. Instead, we can decide afterward how many topics seem realistic.

Note: The final model used for analysis uses **ALL 541** topics, with no reduction.

In [ ]:
# Function to reduce topics
reduce_topics = 0 # change to 1 if you want to reduce topics
new_number_of_topics = 25 # Can be changed based on analysis

if reduce_topics:
  print("Topic Reduction underway")
  loaded_model.reduce_topics(documents, nr_topics=new_number_of_topics)
  print("Topic Reduction completed")
# merge the topic based on their similarity based on the distance between their centroids in the embedding space.

In [ ]:
updated_model_path = model_path + '/updated'
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
loaded_model.save(updated_model_path, serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

In [ ]:
topics = loaded_model.topics_
len(topics)
topic_labels = loaded_model.generate_topic_labels()
topic_labels

['-1_love_im_dont',
 '0_nigga_niggas_bitch',
 '1_jesus_praise_lord',
 '2_body_girl_baby',
 '3_dance_funky_dance dance',
 '4_ayy ayy_change_long sentiment',
 '5_christmas_santa_year',
 '6_wi_ye_sae',
 '7_mi_dem_yuh',
 '8_shes_shell_borderlines',
 '9_money_money money_dub dub',
 '10_love_make love_baby',
 '11_tears_heart_wish',
 '12_blues_worried_blues blues',
 '13_id_world world_wouldnt',
 '14_shes_shes got_cameron',
 '15_sea_sail_ocean',
 '16_mama_mama dont_woman woman',
 '17_black_white_white man',
 '18_feature_remix_demo',
 '19_shadow_light_moon',
 '20_christ_law_unto',
 '21_fool_fools_im fool',
 '22_smoke_weed_marijuana',
 '23_vote_skeleton_reron',
 '24_raining_rain_raining raining',
 '25_oliver_sea_moira',
 '26_la la_la_lalala',
 '27_johanna_mortimer_thou',
 '28_spring_summer_autumn',
 '29_album_songs_song',
 '30_angel_youre angel_angel angel',
 '31_sudden felt_moving sound_felt body',
 '32_rattle roll_shake rattle_eat',
 '33_heartache_bah_bah bah',
 '34_drink_drink drink_beer',
 '

In [ ]:
len(generated_topics)

537553

In [ ]:
prob

array([0.6098172 , 0.6756907 , 0.6523981 , ..., 0.59119713, 0.6310166 ,
       0.7728381 ], dtype=float32)

In [ ]:
def generate_topic_label_dictionary():
  topic_label_dictionary = {}
  topic_label_list = [{label.split('_')[0]: label.split('_', 1)[1]} for label in topic_labels]

  for item in topic_label_list:
    topic_label_dictionary.update(item)
  return topic_label_dictionary

topic_label_dictionary = generate_topic_label_dictionary()
print(topic_label_dictionary)

{'-1': 'love_im_dont', '0': 'nigga_niggas_bitch', '1': 'jesus_praise_lord', '2': 'body_girl_baby', '3': 'dance_funky_dance dance', '4': 'ayy ayy_change_long sentiment', '5': 'christmas_santa_year', '6': 'wi_ye_sae', '7': 'mi_dem_yuh', '8': 'shes_shell_borderlines', '9': 'money_money money_dub dub', '10': 'love_make love_baby', '11': 'tears_heart_wish', '12': 'blues_worried_blues blues', '13': 'id_world world_wouldnt', '14': 'shes_shes got_cameron', '15': 'sea_sail_ocean', '16': 'mama_mama dont_woman woman', '17': 'black_white_white man', '18': 'feature_remix_demo', '19': 'shadow_light_moon', '20': 'christ_law_unto', '21': 'fool_fools_im fool', '22': 'smoke_weed_marijuana', '23': 'vote_skeleton_reron', '24': 'raining_rain_raining raining', '25': 'oliver_sea_moira', '26': 'la la_la_lalala', '27': 'johanna_mortimer_thou', '28': 'spring_summer_autumn', '29': 'album_songs_song', '30': 'angel_youre angel_angel angel', '31': 'sudden felt_moving sound_felt body', '32': 'rattle roll_shake rattl

In [ ]:
df['has_lyrics'] = ~df['lyrics'].isna() # flag the rows that have lyrics
# documents = df[df['has_lyrics']]['cleaned_lyrics'].astype(str).tolist()
df.loc[~df['has_lyrics'], 'topic'] = "No Lyrics"  # e.g., -1 or "No Lyrics"
# Assign topics only to rows where 'has_lyrics' is True
df.loc[df['has_lyrics'], 'topic'] = generated_topics
df.loc[df['has_lyrics'], 'topic_prob'] = prob

<ipython-input-32-0b3760e3b1ed>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df['has_lyrics'], 'topic'] = generated_topics


In [ ]:
# loaded_model.visualize_topics()

In [ ]:
# loaded_model.visualize_heatmap()

In [ ]:
# Group by topics and gender, and count occurrences, aggregate lyrices by topic and gender
topic_gender_distribution = df.groupby(['topic', 'gender']).size().unstack(fill_value=0)

#  normalize the counts to compare proportions rather than raw counts
topic_gender_distribution_normalized = topic_gender_distribution.div(topic_gender_distribution.sum(axis=1), axis=0)

In [ ]:
# Aggregate lyrics by topic and gender, groups the DataFrame by topic and gender and then concatenates all lyrics within each group.
aggregated_lyrics = df.groupby(['topic', 'gender'])['cleaned_lyrics'].apply(lambda x: ' '.join(x)).reset_index()
# Extract embeddings for each group
# embeddings = loaded_model._extract_embeddings(documents) # not recommended to use this method since it is an internal method and its use is not for standard operations, this functions also use internal sentence transformers

KeyboardInterrupt: ignored

In [ ]:
aggregated_lyrics

,topic,gender,cleaned_lyrics
0,-1,Female,"{entire song set to the melody of R. Kelly's ""..."
1,-1,Male,\n(*harmonizing*)\nAll my love is all I have\n...
2,0,Female,\nLove me when I'm broke or when I'm filthy fu...
3,0,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197..."
4,1,Female,\nThe conquering lion\nShall break every chain...
...,...,...,...
1079,539,Male,\nSome people was born in heaven\nWith a silve...
1080,540,Female,"I find\nIf it's not the world, its the people\..."
1081,540,Male,What'll I do?\nWhen you are far away\nAnd I am...
1082,541,Female,\nHeaven please send to all mankind\nUnderstan...


In [ ]:
df['embedding'] = list(embeddings)

In [ ]:
aggregated_lyrics.head()
agg_labels = []
df_topic_labels = []

# Adding topic label to aggregated_lyrics and df
for i in aggregated_lyrics['topic']:
  agg_labels.append(topic_label_dictionary[str(i)])

aggregated_lyrics['topic_label'] = agg_labels

for i in df['topic']:
  df_topic_labels.append(topic_label_dictionary[str(i)])

df['topic_label'] = df_topic_labels

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics,has_lyrics,topic,topic_prob,topic_label
0,0,0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",en,Male,"\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa ...",True,7,0.609817,mi_dem_yuh
1,1,1,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,en,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",True,0,0.675691,nigga_niggas_bitch
2,2,2,Family Ties,rap,Cam'ron,2004,41960,"{""Cam\\'ron"",""Lady Wray""}","[Verse 1: Cam'ron]\nKilla, Dipset\nMan I spit ...",en,Male,"\nKilla, Dipset\nMan I spit that pimp talk, yo...",True,434,0.652398,bicyclette_ma bicyclette_bicyclette ma
3,3,3,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,en,Male,\nAy yo you wonder who I are\nI guzzle up at t...,True,434,0.742198,bicyclette_ma bicyclette_bicyclette ma
4,4,4,Lord You Know,rap,Cam'ron,2004,11882,"{""Cam\\'ron"",""Juelz Santana"",Jaheim}","[Chorus: Jaheim]\nNow Lord you know, just how ...",en,Male,"\nNow Lord you know, just how hard I try\nTo l...",True,398,0.737625,dog status_movin rank_status im


In [ ]:
# Save the CSVs with the results

results_csv_path = model_path + "/topics_for_bias_analysis.csv"
df.to_csv(results_csv_path)

agg_lyrics_path = model_path + "/aggregated_lyrics.csv"
aggregated_lyrics.to_csv(agg_lyrics_path)